In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
import sklearn as sk
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'],format='%m/%d/%y')

#plotting the prices againts the dates to analse the trend and pattern 
plt.style.use('dark_background')
plt.figure(figsize=(10,6))
sns.lineplot(data=df,x='Dates',y='Prices',label='Price',color='cyan')
plt.xticks(rotation=45) 
plt.title('Natural Gas Prices over the years',fontsize=20)
plt.xlabel('Dates',fontsize=15)
plt.ylabel('Prices',fontsize=15)
plt.legend()

df['month'] = df['Dates'].dt.month
monthly_avg = df.groupby('month')['Prices'].mean()

plt.figure(figsize=(10,6))
plt.style.use('dark_background')

sns.lineplot(x=monthly_avg.index,y=monthly_avg.values,label='Price',color='cyan')
plt.title('Monthly Avetage Prices',fontsize=20)
plt.xlabel('Dates',fontsize=15)
plt.ylabel('Average Prices',fontsize=15)


In [ ]:
# using linear regression over the time series and adate points
df['Daysfromstart'] = (df['Dates'] - df['Dates'].min()).dt.days

#Linear regression
x_linear = df['Daysfromstart']
y =  df['Prices']

linear_model = sk.linear_model.LinearRegression()
linear_model.fit(x_linear.values.reshape(-1,1),y)
linear_pred = linear_model.predict(x_linear.values.reshape(-1,1))

#polynotmial regression (degree 2)
poly = sk.preprocessing.PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(x_linear.values.reshape(-1,1))
poly_model = sk.linear_model.LinearRegression()
poly_model.fit(x_poly,y)
poly_pred = poly_model.predict(x_poly)

# seasonal Regresssion
df['month_angle'] = 2*np.pi*df['month']/12
x_seasonal = np.column_stack([df['Daysfromstart'],np.sin(df['month_angle']),np.cos(df['month_angle'])])

seasonal_model = sk.linear_model.LinearRegression()
seasonal_model.fit(x_seasonal,y)
seasonal_pred = seasonal_model.predict(x_seasonal)

r2={}
r2['Linear Regression'] = r2_score(y,linear_pred)
r2['Linear Regression'+'(Degree 2)'] = r2_score(y,poly_pred)
r2['seasonal Regression'] = r2_score(y,seasonal_pred)

def predictor(date,model='seasonal'):
    date = pd.to_datetime(date,format='%m/%d/%y')
    days_from_start = (date - df['Dates'].min()).days
    month = date.month
    month_angle = 2*np.pi*month/12

    if model == 'linear':
        pred = linear_model.predict(np.array([[days_from_start]]))
    elif model == 'poly':
        days_poly = poly.transform(np.array([[days_from_start]]))
        pred = poly_model.predict(days_poly)
    elif model == 'seasonal':
        seasonal_features = np.array([[days_from_start,np.sin(month_angle),np.cos(month_angle)]])
        pred = seasonal_model.predict(seasonal_features)
    else:
        raise ValueError("Model not recognized. Choose from 'linear', 'poly', or 'seasonal'.")
    
    return pred[0]


np.float64(12.98685143842427)